In [1]:
import numpy as np
import pandas as pd
from datetime import timedelta

In [2]:
# Load in the data
# Field notes
s_meas = pd.read_excel('C:/Users/Owner/Documents/School/CMPT898/170621_LDP_Sutherland.xlsx', sheetname = 'Measurements')
s_meas = s_meas.dropna(subset = ['Days to Emergence (days)'])
r_meas = pd.read_excel('C:/Users/Owner/Documents/School/CMPT898/170620_LDP_Rosthern.xlsx', sheetname = 'Measurements')
r_meas = r_meas.dropna(subset = ['Days to Emergence'])
# Seed weight
seed_weight = pd.read_excel('C:/Users/Owner/Documents/School/CMPT898/170621-1000sdwt (2016 set up).xlsx', sheetname = 'Inventory')
# Datalogger from both sites, have to cut off some of the later dates as the datalogger output has poor formatting
# Excel data is M/D/Y and if M & D < 12, the import statement will assume D/M/Y. Luckily all datalogger data is after the 12th and before the end of the month 
# Drop first row because it's just instrument specs
s_datalogger = pd.read_excel('C:/Users/Owner/Documents/School/CMPT898/AGILE_LDP_2017-06-23_olddataloggers.xlsx', sheetname = 0, skip_footer = 603 , skiprows = [0])
r_datalogger = pd.read_excel('C:/Users/Owner/Documents/School/CMPT898/AGILE_LDP_2017-06-23_olddataloggers.xlsx', sheetname = 1, skip_footer = 545, skiprows = [0])
s_datalogger.columns = ['drop', 'Datetime', 'Temp (F)', 'drop', 'drop', 'drop', 'drop', 'drop', 'drop']
s_datalogger = s_datalogger.drop('drop', axis = 1)
s_datalogger['Datetime'] = pd.to_datetime(s_datalogger['Datetime'])
r_datalogger.columns = ['drop', 'Datetime', 'Temp (C)', 'drop', 'drop', 'drop', 'drop', 'drop', 'drop']
r_datalogger = r_datalogger.drop('drop', axis = 1)
r_datalogger['Datetime'] = pd.to_datetime(r_datalogger['Datetime'])
# Make an array of the cumulative rainfall - data from environment canada Saskatoon RCS station
# s_precip starts on 05-04, ends on 05/23
s_precip = np.array([0, 0, 0, 17.5, 5.2, 0, 0, 0, 0, 5.9, 13.0, 0, 0, 0, 0, 0, 0, 0, 0]).cumsum()
# r_precip starts on 05-19, ends on 05-31
r_precip = np.array([0, 0, 0, 0, 0, 0, 2.6, 0, 0, 0, 0, 0, 0, 0]).cumsum()

In [3]:
# What are the dates in question?
# plant date in datetime
s_plant_date = s_meas['Planting Date\n(date)'].iloc[0]
r_plant_date = pd.to_datetime(r_meas['Planting Date\n(date)']).iloc[0]
# days to emerge from planting as numbers
s_days_to_emerge = np.unique(s_meas['Days to Emergence (days)'].values)
r_days_to_emerge = np.unique(r_meas['Days to Emergence'].values)
# add one to last day to get everything the day of as well
s_last_day_idx = s_plant_date + timedelta(days = np.max(s_days_to_emerge) + 1)
r_last_day_idx = r_plant_date + timedelta(days = np.max(r_days_to_emerge) + 1)

In [4]:
# Drop everything that is after emergence
s_datalogger = s_datalogger[s_datalogger['Datetime'] < s_last_day_idx]
r_datalogger = r_datalogger[r_datalogger['Datetime'] < r_last_day_idx]
r_datalogger = r_datalogger.dropna()
# convert from F to C in sutherland
s_datalogger['Temp (F)'] = (s_datalogger['Temp (F)'] - 32) * 5 / 9
s_datalogger.columns = ['Datetime', 'Temp (C)']

In [5]:
# Get the GDD from the datalogger
# pop in a few extra bits at the beginning to give each day 24 readings, without significantly adjusting the mean (the mean is high because the values are just the afternoon)
s_GDD_datalogger = np.mean(np.reshape(np.pad(s_datalogger['Temp (C)'].values, (12,0), 'constant', constant_values = 19.2), (-1, 24)), axis = 1) - 5
r_GDD_datalogger = np.mean(np.reshape(np.pad(r_datalogger['Temp (C)'].values, (11,0), 'constant', constant_values = 19.2), (-1, 24)), axis = 1) - 5
# Append environment canada data to the sutherland set, starting at 05-04 up to (not including) 05-19
s_GDD_datalogger = np.insert(s_GDD_datalogger, 0, [9.3, 12.2, 17.3, 9.8, 4.5, 4.0, 5.9, 9.4, 6.9, 6.0, 5.8, 5.4, 2.2, 1.6, 2.7])

In [6]:
# Make the cumulative sum
s_GDD = s_GDD_datalogger.cumsum()
r_GDD = r_GDD_datalogger.cumsum()

In [7]:
# daylength from environment canada sunrise/sunset calculator - advanced options and sun angle - day illumination
# Rosthern 106 20' W 52 40' N 05-04 up to 05-23
r_daylength = np.array([16.01, 16.06, 16.10, 16.15, 16.19, 16.24, 16.28, 16.32, 16.36, 16.40, 16.44, 16.47])
# Sutherland - Saskatoon 05-19 up to 05-30
s_daylength = np.array([15.13, 15.19, 15.24, 15.30, 15.36, 15.41, 15.47, 15.52, 15.57, 15.63, 15.68, 15.73, 15.78, 15.83, 15.88, 15.93, 15.97, 16.02])

In [14]:
s_data = pd.DataFrame()
s_data = s_data.assign(Entry = s_meas.sort_values('Entry')['Entry'].astype(int))
s_data = s_data.assign(Line = s_meas.sort_values('Entry')['Name'])
s_data['Line'] = s_data['Line'].replace(' ', '-', regex = True)
s_data = s_data.assign(DTE = s_meas.sort_values('Entry')['Days to Emergence (days)'].astype(int))
s_data = s_data.assign(GDD = s_GDD[s_meas.sort_values('Entry')['Days to Emergence (days)'].astype(int)])
s_data = s_data.assign(Precip = s_precip[s_meas.sort_values('Entry')['Days to Emergence (days)'].astype(int)])
s_data = s_data.assign(Seedwt = seed_weight['1000 Sdwt'][s_meas.sort_values('Entry')['Entry'].values.astype(int) - 1].values)
s_data = s_data.assign(Day = s_daylength[(s_meas.sort_values('Entry')['Days to Emergence (days)'] / 2).astype(int)])
s_data = s_data.assign(Location = 'Sutherland')
s_data = s_data.assign(Year = '2017')
s_data.columns = ['Entry', 'Line', 'DTE', 'GDD [C]', 'Precipitation [mm]', 'Seed 1000wt [g]', 'Day Length [hr]', 'Location', 'Year']

In [15]:
r_data = pd.DataFrame()
r_data = r_data.assign(Entry = r_meas.sort_values('Entry')['Entry'].astype(int))
r_data = r_data.assign(Line = r_meas.sort_values('Entry')['Name'])
r_data['Line'] = r_data['Line'].replace(' ', '-', regex = True)
r_data = r_data.assign(DTE = r_meas.sort_values('Entry')['Days to Emergence'].astype(int))
r_data = r_data.assign(GDD = r_GDD[r_meas.sort_values('Entry')['Days to Emergence'].astype(int)])
r_data = r_data.assign(Precip = r_precip[r_meas.sort_values('Entry')['Days to Emergence'].astype(int)])
r_data = r_data.assign(Seedwt = seed_weight['1000 Sdwt'][r_meas.sort_values('Entry')['Entry'].values.astype(int) - 1].values)
r_data = r_data.assign(Day = r_daylength[(r_meas.sort_values('Entry')['Days to Emergence'] / 2).astype(int)])
r_data = r_data.assign(Location = 'Rosthern')
r_data = r_data.assign(Year = '2017')
r_data.columns = ['Entry', 'Line', 'DTE', 'GDD [C]', 'Precipitation [mm]', 'Seed 1000wt [g]', 'Day Length [hr]', 'Location', 'Year']

In [16]:
r_data.to_csv('Rosthern Climate Data 2017.csv', index = False)
s_data.to_csv('Sutherland Climate Data 2017.csv', index = False)